In [1]:
import pandas as pd

In [2]:
! pip install google-cloud
! pip install google-cloud-bigquery[pandas]
! pip install google-cloud-storage

In [3]:
%load_ext google.cloud.bigquery

In [4]:
%%bigquery scores
select * from meetfresh.ft_customer_ingredient_ratings

Query is running:   0%|          |

Downloading:   0%|          |

In [5]:
scores.head()

,customer_id,ingredient_id,ingredient_name,meetfresh_rating
0,49237,B15,Tofu Pudding,1.0
1,172644,B15,Tofu Pudding,1.0
2,39930,B15,Tofu Pudding,1.0
3,207442,B15,Tofu Pudding,1.0
4,78835,B15,Tofu Pudding,1.0


In [6]:
scores[['ingredient_id', 'ingredient_name', 'meetfresh_rating']].describe()

,ingredient_id,ingredient_name,meetfresh_rating
count,500030,500030,500030
unique,23,23,4
top,TP108,Taro Balls,2.0
freq,85653,85653,217746


In [7]:
scores['meetfresh_rating'].unique()

array(['1.0', '2.0', '3.0', 'meetfresh_rating'], dtype=object)

In [8]:
scores[scores['meetfresh_rating'] == 'meetfresh_rating'] 

,customer_id,ingredient_id,ingredient_name,meetfresh_rating
500029,customer_id,ingredient_id,ingredient_name,meetfresh_rating


In [9]:
scores.tail()

,customer_id,ingredient_id,ingredient_name,meetfresh_rating
500025,12651,UNK1002,Almond Flakes,3.0
500026,38856,UNK1002,Almond Flakes,3.0
500027,226170,UNK1002,Almond Flakes,3.0
500028,70724,UNK1002,Almond Flakes,3.0
500029,customer_id,ingredient_id,ingredient_name,meetfresh_rating


In [10]:
scores = scores.iloc[:len(scores)-1, :]

In [11]:
len(scores['customer_id'].unique())

155594

In [23]:
scores = scores.astype({"meetfresh_rating": float, "customer_id": int})

In [47]:
temp = scores[['customer_id', 'ingredient_id']].groupby('customer_id').count()

In [48]:
temp[temp['ingredient_id']>22]

,ingredient_id
customer_id,
741,29
1846,25
2531,28
2547,31
3140,25
...,...
267772,29
268239,25
268356,23


In [46]:
scores[scores['customer_id'] == 270887]

,customer_id,ingredient_id,ingredient_name,meetfresh_rating
28415,270887,TP011,Boba,1.0
112454,270887,TP012,Mini Q (Mini Taro Ball),3.0
150210,270887,TP013,Red Beans,3.0
151899,270887,TP013,Red Beans,3.0
153074,270887,TP013,Red Beans,3.0
210361,270887,TP015,Grass Jelly,3.0
233309,270887,TP016,Peanuts,1.0
240194,270887,TP016,Peanuts,3.0
240366,270887,TP016,Peanuts,3.0
241020,270887,TP016,Peanuts,3.0


In [13]:
scores[['ingredient_name', 'meetfresh_rating']].groupby('ingredient_name').mean().sort_values("meetfresh_rating")

,meetfresh_rating
ingredient_name,
Coco Mochi,1.589320
Almond Flakes,1.644866
Sago,1.652605
Q Mochi,1.659941
Lychee Jelly,1.671277
Black Sugar Boba,1.690247
Peanuts,1.691534
Caramel Pudding,1.698761
Tofu Pudding,1.717668


In [24]:
matrix = pd.DataFrame(columns=scores['ingredient_name'].unique(), index=scores['customer_id'].unique())

In [28]:
for index, row in scores.iterrows():
    matrix.loc[row['customer_id'],row['ingredient_name']] = row['meetfresh_rating']
# ~ 1min

In [35]:
matrix.head()

,Tofu Pudding,Rice Balls,Boba,Mini Q (Mini Taro Ball),Red Beans,Grass Jelly,Peanuts,Black Sugar Boba,Melon Jelly,Lychee Jelly,...,Potaro Balls,Caramel Pudding,Sesame Rice Balls,Taro Balls,Almond Pudding,Taro Paste,Q Mochi,Coco Mochi,Sago,Almond Flakes
49237,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,NaN,2.0,NaN,NaN,NaN,NaN
172644,1.0,NaN,3.0,3.0,2.0,3.0,3.0,NaN,NaN,2.0,...,2.0,NaN,3.0,1.0,2.0,2.0,NaN,NaN,3.0,NaN
39930,1.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.0,NaN,1.0,3.0,NaN,NaN,NaN
207442,2.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,NaN,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
78835,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
from google.cloud import storage
import os
import pandas as pd

client = storage.Client()
bucket = client.get_bucket('meet-fresh-recommendation-system-data')

bucket.blob('meetfresh_customer_item_rating_matrix.csv').upload_from_string(matrix.to_csv(index=True), 'text/csv')